# Notebook 06 — COVID-19 Sub-Period Analysis

## How Did COVID-19 Reshape Credit Risk Dynamics?

This notebook splits the data into three sub-periods and compares OU parameters and CRI across them.

| Period | Dates | Label | Obs |
|--------|-------|-------|-----|
| Pre-COVID | Jan 2013 – Dec 2019 | Baseline | 84 |
| COVID | Jan 2020 – Dec 2021 | Shock | 24 |
| Post-COVID | Jan 2022 – Dec 2025 | Recovery | 48 |

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats as sp_stats
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.figsize': (12, 6), 'figure.dpi': 150, 'savefig.dpi': 300,
    'font.size': 11, 'axes.titlesize': 14, 'axes.labelsize': 12,
    'legend.fontsize': 10, 'font.family': 'serif'
})
print('Libraries loaded.')

In [ ]:
# ─── Load Data ───────────────────────────────────────────────────────────────
usd = pd.read_csv('../data/processed/spreads_usd_new_amount.csv', parse_dates=['date'], index_col='date')
khr = pd.read_csv('../data/processed/spreads_khr_new_amount.csv', parse_dates=['date'], index_col='date')
dt = 1/12

# OU estimation function
def ou_neg_log_likelihood(params, data, dt):
    kappa, theta, sigma = params
    if kappa <= 0 or sigma <= 0:
        return 1e10
    n = len(data) - 1
    exp_kdt = np.exp(-kappa * dt)
    m = theta + (data[:-1] - theta) * exp_kdt
    v = (sigma**2 / (2 * kappa)) * (1 - np.exp(-2 * kappa * dt))
    if v <= 0:
        return 1e10
    residuals = data[1:] - m
    ll = -0.5 * n * np.log(2 * np.pi) - 0.5 * n * np.log(v) - 0.5 * np.sum(residuals**2) / v
    return -ll

def estimate_ou(data, dt):
    best_nll = np.inf
    best_x = None
    for k0 in [0.5, 1.0, 2.0, 5.0, 10.0, 20.0]:
        result = minimize(ou_neg_log_likelihood,
                         [k0, np.mean(data), np.std(np.diff(data))*np.sqrt(12)],
                         args=(data, dt), method='Nelder-Mead',
                         options={'maxiter': 50000, 'xatol': 1e-10, 'fatol': 1e-10})
        if result.fun < best_nll and result.x[0] > 0 and result.x[2] > 0:
            best_nll = result.fun
            best_x = result.x
    return {'kappa': best_x[0], 'theta': best_x[1], 'sigma': best_x[2],
            'half_life': np.log(2) / best_x[0] * 12}

print('Functions defined.')

In [ ]:
# ─── Sub-Period Split ────────────────────────────────────────────────────────
periods = {
    'Pre-COVID': ('2013-01-01', '2019-12-31'),
    'COVID':     ('2020-01-01', '2021-12-31'),
    'Post-COVID':('2022-01-01', '2025-12-31')
}

results = []
for pname, (start, end) in periods.items():
    mask_usd = (usd.index >= start) & (usd.index <= end)
    mask_khr = (khr.index >= start) & (khr.index <= end)
    
    data_usd = usd[mask_usd]['spread'].values
    data_khr = khr[mask_khr]['spread'].values
    
    print(f'\n── {pname} ({len(data_usd)} obs) ──')
    
    est_usd = estimate_ou(data_usd, dt)
    est_khr = estimate_ou(data_khr, dt)
    
    # Compute average crisis probability for each
    Sc_usd = np.percentile(usd['spread'].values, 95)  # full-sample threshold
    Sc_khr = np.percentile(khr['spread'].values, 95)
    
    def avg_crisis_prob(data, kappa, theta, sigma, Sc):
        exp_kdt = np.exp(-kappa * dt)
        v = (sigma**2 / (2 * kappa)) * (1 - np.exp(-2 * kappa * dt))
        probs = []
        for t in range(1, len(data)):
            m_t = theta + (data[t-1] - theta) * exp_kdt
            probs.append(1 - sp_stats.norm.cdf(Sc, loc=m_t, scale=np.sqrt(v)))
        return np.mean(probs)
    
    prob_usd = avg_crisis_prob(data_usd, est_usd['kappa'], est_usd['theta'], est_usd['sigma'], Sc_usd)
    prob_khr = avg_crisis_prob(data_khr, est_khr['kappa'], est_khr['theta'], est_khr['sigma'], Sc_khr)
    
    results.append({
        'Period': pname,
        'N': len(data_usd),
        'θ_USD': est_usd['theta'], 'κ_USD': est_usd['kappa'],
        'σ_USD': est_usd['sigma'], 'HL_USD': est_usd['half_life'],
        'CRI_USD': prob_usd,
        'θ_KHR': est_khr['theta'], 'κ_KHR': est_khr['kappa'],
        'σ_KHR': est_khr['sigma'], 'HL_KHR': est_khr['half_life'],
        'CRI_KHR': prob_khr,
    })
    
    print(f'  USD: θ={est_usd["theta"]:.2f}%, κ={est_usd["kappa"]:.2f}, σ={est_usd["sigma"]:.2f}, HL={est_usd["half_life"]:.1f}m')
    print(f'  KHR: θ={est_khr["theta"]:.2f}%, κ={est_khr["kappa"]:.2f}, σ={est_khr["sigma"]:.2f}, HL={est_khr["half_life"]:.1f}m')

In [ ]:
# ─── TABLE 5: COVID Sub-Period Parameter Comparison ──────────────────────────
res_df = pd.DataFrame(results).set_index('Period')

print('\n══════════════════════════════════════════════════════════════════════════════════════════')
print('                  TABLE 5: OU Parameters by Sub-Period')
print('══════════════════════════════════════════════════════════════════════════════════════════')
cols = ['N', 'θ_USD', 'κ_USD', 'σ_USD', 'HL_USD', 'CRI_USD', 'θ_KHR', 'κ_KHR', 'σ_KHR', 'HL_KHR', 'CRI_KHR']
print(res_df[cols].round(4).to_string())
print('══════════════════════════════════════════════════════════════════════════════════════════')
print('HL = Half-life in months')

In [ ]:
# ─── FIGURE 9: Sub-Period Parameter Comparison ───────────────────────────────
fig, axes = plt.subplots(2, 3, figsize=(16, 9))
period_names = [r['Period'] for r in results]
x = np.arange(len(period_names))
width = 0.35

params_to_plot = [
    ('θ (Long-Run Mean, %)', 'θ_USD', 'θ_KHR'),
    ('κ (Mean Reversion Speed)', 'κ_USD', 'κ_KHR'),
    ('σ (Volatility)', 'σ_USD', 'σ_KHR'),
    ('Half-Life (months)', 'HL_USD', 'HL_KHR'),
    ('Avg Crisis Probability', 'CRI_USD', 'CRI_KHR'),
]

for idx, (title, col_usd, col_khr) in enumerate(params_to_plot):
    ax = axes.flat[idx]
    vals_usd = [r[col_usd] for r in results]
    vals_khr = [r[col_khr] for r in results]
    
    bars1 = ax.bar(x - width/2, vals_usd, width, label='USD', color='#1565C0', alpha=0.85)
    bars2 = ax.bar(x + width/2, vals_khr, width, label='KHR', color='#C62828', alpha=0.85)
    
    ax.set_title(title, fontweight='bold', fontsize=11)
    ax.set_xticks(x)
    ax.set_xticklabels(period_names, fontsize=9)
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3, axis='y')
    
    # Value labels
    for bars in [bars1, bars2]:
        for bar in bars:
            h = bar.get_height()
            ax.annotate(f'{h:.2f}', xy=(bar.get_x() + bar.get_width()/2, h),
                       xytext=(0, 2), textcoords='offset points',
                       ha='center', va='bottom', fontsize=7)

# Hide empty subplot
axes.flat[5].set_visible(False)

fig.suptitle('Figure 9: OU Parameters Across COVID Sub-Periods',
             fontweight='bold', fontsize=14, y=1.01)
plt.tight_layout()
plt.savefig('../figures/fig9_covid_comparison.png', dpi=300, bbox_inches='tight')
plt.show()
print('Saved: fig9_covid_comparison.png')

In [ ]:
# ─── Key COVID Analysis Questions ────────────────────────────────────────────
pre = results[0]
covid = results[1]
post = results[2]

print('\n═══════════════════════════════════════════════════════════════')
print('          COVID-19 Impact Analysis — Key Findings')
print('═══════════════════════════════════════════════════════════════')

print(f'\n1. Did volatility (σ) spike during COVID?')
print(f'   USD: Pre={pre["σ_USD"]:.3f} → COVID={covid["σ_USD"]:.3f} → Post={post["σ_USD"]:.3f}')
print(f'   KHR: Pre={pre["σ_KHR"]:.3f} → COVID={covid["σ_KHR"]:.3f} → Post={post["σ_KHR"]:.3f}')

print(f'\n2. Which currency was hit harder?')
usd_sigma_change = (covid['σ_USD'] - pre['σ_USD']) / pre['σ_USD'] * 100
khr_sigma_change = (covid['σ_KHR'] - pre['σ_KHR']) / pre['σ_KHR'] * 100
print(f'   USD σ changed by {usd_sigma_change:+.1f}% during COVID')
print(f'   KHR σ changed by {khr_sigma_change:+.1f}% during COVID')

print(f'\n3. Did post-COVID parameters return to pre-COVID levels?')
for param, label in [('θ_USD', 'θ_USD'), ('θ_KHR', 'θ_KHR'), ('σ_USD', 'σ_USD'), ('σ_KHR', 'σ_KHR')]:
    recovery = (post[param] - pre[param]) / pre[param] * 100
    status = '≈ returned' if abs(recovery) < 15 else 'shifted'
    print(f'   {label}: Pre={pre[param]:.3f}, Post={post[param]:.3f} ({recovery:+.1f}%) → {status}')

print(f'\n4. Mean reversion speed (κ) changes:')
print(f'   USD: Pre={pre["κ_USD"]:.3f} → COVID={covid["κ_USD"]:.3f} → Post={post["κ_USD"]:.3f}')
print(f'   KHR: Pre={pre["κ_KHR"]:.3f} → COVID={covid["κ_KHR"]:.3f} → Post={post["κ_KHR"]:.3f}')
print(f'═══════════════════════════════════════════════════════════════')

---
## Summary

The COVID sub-period analysis reveals the structural impact of the pandemic on credit risk dynamics in both currency segments. Key insights include how volatility, mean reversion speed, and equilibrium spreads shifted during and after the crisis, and whether these shifts were temporary or persistent.